In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


X = pd.read_csv('D:/SDE/1_SDEisotropicDL/CollectData/0_DataSet/DataSetFeatureEngg.txt',header=None)
y = pd.read_excel('D:/SDE/1_SDEisotropicDL/CollectData/2_Labels/labelsD.xlsx',header=None)

y1 = y.iloc[:,0:1];
y1 = np.array(y1).ravel()

y2 = y.iloc[:,1:2];
y2 = np.array(y2).ravel()

In [2]:
#--------------------------------R^2 coefficient for variance------------
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [3]:
def SVMregress(X, y, svrtype):
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    from sklearn.svm import SVR
    
    if (svrtype == 1): # SVR with Radial bias function
        svr_rbf = SVR(kernel='rbf', C=10, gamma=0.1, epsilon=.1)
        history = svr_rbf.fit(X_train_scaled, y_train)
    elif (svrtype == 2): # SVR with linear function
        svr_lin = SVR(kernel='linear', C=10, gamma='auto')
        history = svr_lin.fit(X_train_scaled, y_train)
    elif (svrtype == 3):# SVR with polynomial function
        svr_poly = SVR(kernel='poly', C=10, gamma='auto', degree=3, epsilon=.1, coef0=1)
        history = svr_poly.fit(X_train_scaled, y_train)
    
    ypred = history.predict(X_test_scaled)
    
    diff = ypred - y_test
    percentDiff = (diff / y_test) * 100
    absPercentDiff = np.abs(percentDiff)
    APD = pd.DataFrame(absPercentDiff)
    APD = round(APD,2)
    
    return np.array(APD)

In [4]:
def LinRegress(X, y, lintype):
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    
    # fit
    if (lintype == 1): # Linear Regression
        linreg = LinearRegression().fit(X_train_scaled, y_train)
        history = linreg
    elif (lintype == 2): # Ridge Regression
        linridge = Ridge(alpha = 20.0).fit(X_train_scaled, y_train)
        history = linridge
    elif (lintype == 3):# Lasso Regression
        linlasso = Lasso(alpha = 2.0, max_iter = 10000).fit(X_train_scaled, y_train)
        history = linlasso
        
    # predict
    y_pred = history.predict(X_test_scaled)
    diff = y_pred - y_test
    percentDiff = (diff / y_test) * 100
    absPercentDiff = np.abs(percentDiff)
    APD = pd.DataFrame(absPercentDiff)
    APD = round(APD,2)
    
    return np.array(APD)

In [5]:
def RandomForestRegress(X, y):
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    from sklearn.ensemble import RandomForestRegressor
    
    # fit
    RFregressor = RandomForestRegressor(n_estimators = 30, random_state = 42)
    history = RFregressor.fit(X_train_scaled, y_train)   
    
    # predict
    y_pred = history.predict(X_test_scaled)
    diff = y_pred - y_test
    percentDiff = (diff / y_test) * 100
    absPercentDiff = np.abs(percentDiff)
    APD = pd.DataFrame(absPercentDiff)
    APD = round(APD,2)
    
    return np.array(APD)

In [6]:
def ANNregress(X, y):
    
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Activation, Dropout
    from tensorflow.keras.optimizers import SGD, Adam, RMSprop
    
    from tensorflow.compat.v1 import ConfigProto 
    from tensorflow.compat.v1 import InteractiveSession
    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)
    
    X = np.array(X)
    y = np.array(y)
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)
    
    
    # architecture of model
    model = Sequential()
    model.add(Dense(2048, input_dim = X.shape[1], activation='relu'))
    model.add(Dense(2, activation='linear'))
    
    # compile model
    model.compile(loss='mse', optimizer=Adam(lr=1e-5), metrics=['mae', 'mape', r_square])
    
    # fit model
    model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 16, epochs=500, shuffle = True)
    
    # predict
    y_pred = model.predict(X_test)
    diff = y_pred - y_test
    percentDiff = (diff / y_test) * 100
    absPercentDiff = np.abs(percentDiff)
    APD = pd.DataFrame(absPercentDiff)
    APD = round(APD,2)
    
    return np.array(APD)

In [7]:
# APD for location for SVM = RBF 
APDxSVM1 = SVMregress(X, y1, svrtype = 1)
# APD for location for SVM = Linear
APDxSVM2 = SVMregress(X, y1, svrtype = 2)
# APD for location for SVM = polynomial
APDxSVM3 = SVMregress(X, y1, svrtype = 3)

In [8]:
# APD for length for SVM = RBF 
APDaSVM1 = SVMregress(X, y2, svrtype = 1)
# APD for length for SVM = Linear
APDaSVM2 = SVMregress(X, y2, svrtype = 2)
# APD for length for SVM = polynomial
APDaSVM3 = SVMregress(X, y2, svrtype = 3)

In [9]:
# APD for location for Linear Regression
APDxLR1 = LinRegress(X, y1, lintype = 1)
# APD for location for Ridge Regression
APDxLR2 = LinRegress(X, y1, lintype = 2)
# APD for location for Lasso polynomial
APDxLR3 = LinRegress(X, y1, lintype = 3)

In [10]:
# APD for length for Linear Regression
APDaLR1 = LinRegress(X, y2, lintype = 1)
# APD for length for Ridge Regression
APDaLR2 = LinRegress(X, y2, lintype = 2)
# APD for length for Lasso polynomial
APDaLR3 = LinRegress(X, y2, lintype = 3)

In [11]:
# APD for location for Random Forest
APDxRF = RandomForestRegress(X, y1)

In [12]:
# APD for length for Random Forest
APDaRF = RandomForestRegress(X, y2)

In [13]:
# APD for location and length for ANN
APDann = ANNregress(X, y)

Using TensorFlow backend.


Train on 2125 samples, validate on 375 samples
Epoch 1/500
2125/2125 [==============================] - 1s 471us/sample - loss: 0.1327 - mae: 0.2558 - mape: 95.6248 - r_square: -0.9199 - val_loss: 0.1242 - val_mae: 0.2494 - val_mape: 90.8224 - val_r_square: -0.8730
Epoch 2/500
2125/2125 [==============================] - 0s 127us/sample - loss: 0.1196 - mae: 0.2389 - mape: 83.4885 - r_square: -0.7292 - val_loss: 0.1108 - val_mae: 0.2308 - val_mape: 78.3780 - val_r_square: -0.6671
Epoch 3/500
2125/2125 [==============================] - 0s 126us/sample - loss: 0.1059 - mae: 0.2192 - mape: 73.2616 - r_square: -0.5280 - val_loss: 0.0969 - val_mae: 0.2100 - val_mape: 69.5672 - val_r_square: -0.4548
Epoch 4/500
2125/2125 [==============================] - 0s 128us/sample - loss: 0.0922 - mae: 0.1984 - mape: 68.6726 - r_square: -0.3251 - val_loss: 0.0832 - val_mae: 0.1887 - val_mape: 65.8972 - val_r_square: -0.2463
Epoch 5/500
2125/2125 [==============================] - 0s 124us/sample - lo

2125/2125 [==============================] - 0s 133us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.2606 - r_square: 0.6256 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 73.8057 - val_r_square: 0.6584
Epoch 39/500
2125/2125 [==============================] - 0s 126us/sample - loss: 0.0252 - mae: 0.1088 - mape: 85.1455 - r_square: 0.6250 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 74.6957 - val_r_square: 0.6584
Epoch 40/500
2125/2125 [==============================] - 0s 126us/sample - loss: 0.0252 - mae: 0.1088 - mape: 85.4826 - r_square: 0.6201 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 74.9284 - val_r_square: 0.6583
Epoch 41/500
2125/2125 [==============================] - 0s 128us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.2506 - r_square: 0.6231 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 74.8941 - val_r_square: 0.6583
Epoch 42/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.8948 - r_square: 0.6260 - v

Epoch 76/500
2125/2125 [==============================] - 0s 121us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.2723 - r_square: 0.6323 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 74.5944 - val_r_square: 0.6587
Epoch 77/500
2125/2125 [==============================] - 0s 132us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.4452 - r_square: 0.6311 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 74.1064 - val_r_square: 0.6587
Epoch 78/500
2125/2125 [==============================] - 0s 127us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.2940 - r_square: 0.6298 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 74.4416 - val_r_square: 0.6587
Epoch 79/500
2125/2125 [==============================] - 0s 129us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.5984 - r_square: 0.6237 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 74.1913 - val_r_square: 0.6586
Epoch 80/500
2125/2125 [==============================] - 0s 122us/sample - loss: 0.0252 - mae: 0.1087 - mape: 84.7741 - r_squar

2125/2125 [==============================] - 0s 121us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.4432 - r_square: 0.6223 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 74.0125 - val_r_square: 0.6589
Epoch 114/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.1428 - r_square: 0.6263 - val_loss: 0.0225 - val_mae: 0.1021 - val_mape: 74.4408 - val_r_square: 0.6590
Epoch 115/500
2125/2125 [==============================] - 0s 126us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.0237 - r_square: 0.6254 - val_loss: 0.0224 - val_mae: 0.1021 - val_mape: 74.7703 - val_r_square: 0.6589
Epoch 116/500
2125/2125 [==============================] - 0s 126us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.7290 - r_square: 0.6272 - val_loss: 0.0224 - val_mae: 0.1021 - val_mape: 73.8906 - val_r_square: 0.6587
Epoch 117/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0252 - mae: 0.1087 - mape: 85.1072 - r_square: 0.6300

...

2125/2125 [==============================] - 0s 121us/sample - loss: 0.0251 - mae: 0.1085 - mape: 85.2989 - r_square: 0.6300 - val_loss: 0.0224 - val_mae: 0.1019 - val_mape: 74.2132 - val_r_square: 0.6603
Epoch 336/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0251 - mae: 0.1085 - mape: 84.9295 - r_square: 0.6270 - val_loss: 0.0224 - val_mae: 0.1019 - val_mape: 74.8472 - val_r_square: 0.6603
Epoch 337/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0251 - mae: 0.1085 - mape: 85.9384 - r_square: 0.6334 - val_loss: 0.0224 - val_mae: 0.1019 - val_mape: 74.6399 - val_r_square: 0.6602
Epoch 338/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0251 - mae: 0.1085 - mape: 85.2073 - r_square: 0.6252 - val_loss: 0.0224 - val_mae: 0.1019 - val_mape: 74.3482 - val_r_square: 0.6601
Epoch 339/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0251 - mae: 0.1085 - mape: 85.3263 - r_square: 0.6272

2125/2125 [==============================] - 0s 125us/sample - loss: 0.0251 - mae: 0.1085 - mape: 85.4434 - r_square: 0.6294 - val_loss: 0.0223 - val_mae: 0.1019 - val_mape: 74.2317 - val_r_square: 0.6605
Epoch 373/500
2125/2125 [==============================] - 0s 129us/sample - loss: 0.0251 - mae: 0.1085 - mape: 85.4885 - r_square: 0.6283 - val_loss: 0.0223 - val_mae: 0.1019 - val_mape: 74.0405 - val_r_square: 0.6605
Epoch 374/500
2125/2125 [==============================] - 0s 122us/sample - loss: 0.0251 - mae: 0.1085 - mape: 85.1460 - r_square: 0.6280 - val_loss: 0.0223 - val_mae: 0.1019 - val_mape: 74.3834 - val_r_square: 0.6604
Epoch 375/500
2125/2125 [==============================] - 0s 126us/sample - loss: 0.0250 - mae: 0.1085 - mape: 85.2267 - r_square: 0.6294 - val_loss: 0.0223 - val_mae: 0.1019 - val_mape: 74.4604 - val_r_square: 0.6605
Epoch 376/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0250 - mae: 0.1085 - mape: 85.3452 - r_square: 0.6292

2125/2125 [==============================] - 0s 124us/sample - loss: 0.0250 - mae: 0.1085 - mape: 84.7856 - r_square: 0.6320 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.6927 - val_r_square: 0.6606
Epoch 410/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0250 - mae: 0.1085 - mape: 85.7655 - r_square: 0.6304 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.5968 - val_r_square: 0.6606
Epoch 411/500
2125/2125 [==============================] - 0s 126us/sample - loss: 0.0250 - mae: 0.1085 - mape: 85.3025 - r_square: 0.6295 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.6663 - val_r_square: 0.6605
Epoch 412/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0250 - mae: 0.1084 - mape: 85.4718 - r_square: 0.6339 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.0827 - val_r_square: 0.6605
Epoch 413/500
2125/2125 [==============================] - 0s 124us/sample - loss: 0.0250 - mae: 0.1084 - mape: 84.9306 - r_square: 0.6272

2125/2125 [==============================] - 0s 130us/sample - loss: 0.0250 - mae: 0.1084 - mape: 85.0303 - r_square: 0.6290 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.2678 - val_r_square: 0.6609
Epoch 447/500
2125/2125 [==============================] - 0s 121us/sample - loss: 0.0250 - mae: 0.1084 - mape: 85.2579 - r_square: 0.6270 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.9621 - val_r_square: 0.6608
Epoch 448/500
2125/2125 [==============================] - 0s 130us/sample - loss: 0.0250 - mae: 0.1084 - mape: 85.7363 - r_square: 0.6288 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.2627 - val_r_square: 0.6610
Epoch 449/500
2125/2125 [==============================] - 0s 120us/sample - loss: 0.0250 - mae: 0.1084 - mape: 84.7881 - r_square: 0.6245 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 75.2336 - val_r_square: 0.6607
Epoch 450/500
2125/2125 [==============================] - 0s 129us/sample - loss: 0.0250 - mae: 0.1084 - mape: 85.7647 - r_square: 0.6250

2125/2125 [==============================] - 0s 126us/sample - loss: 0.0250 - mae: 0.1084 - mape: 85.0751 - r_square: 0.6311 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.5044 - val_r_square: 0.6612
Epoch 484/500
2125/2125 [==============================] - 0s 126us/sample - loss: 0.0250 - mae: 0.1084 - mape: 85.3447 - r_square: 0.6297 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.2909 - val_r_square: 0.6610
Epoch 485/500
2125/2125 [==============================] - 0s 125us/sample - loss: 0.0250 - mae: 0.1084 - mape: 84.9090 - r_square: 0.6283 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 75.0163 - val_r_square: 0.6611
Epoch 486/500
2125/2125 [==============================] - 0s 130us/sample - loss: 0.0250 - mae: 0.1084 - mape: 85.8289 - r_square: 0.6253 - val_loss: 0.0223 - val_mae: 0.1018 - val_mape: 74.4434 - val_r_square: 0.6610
Epoch 487/500
2125/2125 [==============================] - 0s 121us/sample - loss: 0.0250 - mae: 0.1084 - mape: 84.9446 - r_square: 0.6281

In [14]:
import xlsxwriter

workbook = xlsxwriter.Workbook('MLregress_DamLocal.xlsx') 
  
# By default worksheet names in the spreadsheet will be  
# Sheet1, Sheet2 etc., but we can also specify a name. 
worksheet = workbook.add_worksheet("My sheet") 

worksheet.write(0, 0, 'APD (x) in SVM (RBF)') 
worksheet.write(0, 1, 'APD (x) in SVM (Linear)')
worksheet.write(0, 2, 'APD (x) in SVM (Poly3)') 
worksheet.write(0, 3, 'APD (x) in Linear Regression')
worksheet.write(0, 4, 'APD (x) in Ridge Regression') 
worksheet.write(0, 5, 'APD (x) in Lasso Regression')
worksheet.write(0, 6, 'APD (x) in Random Forest')
worksheet.write(0, 7, 'APD (x) in ANN')

worksheet.write(0, 8, 'APD (a) in SVM (RBF)') 
worksheet.write(0, 9, 'APD (a) in SVM (Linear)')
worksheet.write(0, 10, 'APD (a) in SVM (Poly3)') 
worksheet.write(0, 11, 'APD (a) in Linear Regression')
worksheet.write(0, 12, 'APD (a) in Ridge Regression') 
worksheet.write(0, 13, 'APD (a) in Lasso Regression')
worksheet.write(0, 14, 'APD (a) in Random Forest')
worksheet.write(0, 15, 'APD (a) in ANN')

# columns are indexed from 1. 
row = 1
col = 0

# Iterate over the data and write it out row by row. 
for i in range(0, len(APDxSVM1)):
    worksheet.write(row, col, APDxSVM1[i])
    worksheet.write(row, col + 1, APDxSVM2[i]) 
    worksheet.write(row, col + 2, APDxSVM3[i])
    worksheet.write(row, col + 3, APDxLR1[i])
    worksheet.write(row, col + 4, APDxLR2[i]) 
    worksheet.write(row, col + 5, APDxLR3[i])
    worksheet.write(row, col + 6, APDxRF[i])
    worksheet.write(row, col + 7, APDann[i,0])
    
    worksheet.write(row, col + 8, APDaSVM1[i])
    worksheet.write(row, col + 9, APDaSVM2[i]) 
    worksheet.write(row, col + 10, APDaSVM3[i])
    worksheet.write(row, col + 11, APDaLR1[i])
    worksheet.write(row, col + 12, APDaLR2[i]) 
    worksheet.write(row, col + 13, APDaLR3[i])
    worksheet.write(row, col + 14, APDaRF[i])
    worksheet.write(row, col + 15, APDann[i,1])
    
    row += 1
    
workbook.close()

excel = pd.read_excel('MLregress_DamLocal.xlsx')